## styletest

用来进行风格测试的模板，基于提示词和风格词的组合生成图片，然后对比结果。

In [1]:
import json
from urllib import request, parse
import random
import uuid
import urllib.request
import urllib.parse
import io
import os
import time
from PIL.PngImagePlugin import PngInfo
import hashlib
from urllib.error import URLError


## import json file

In [2]:
promptJson = 'nsfw_IPadapter_refiner_api.json'

with open(promptJson) as f:
    prompt = json.load(f)

In [3]:
server_address = "https://2rpluiofff5g94-8188.proxy.runpod.net/"
client_id = str(uuid.uuid4())


In [4]:
def changeImageSize(width, height):
    # 最小的宽高为 896
    limited = 896
    minSize = min(width, height)

    # 缩放宽高，使得最窄的边等于limited
    if minSize > limited:
        scale = minSize / limited
        widthScale = int(width / scale)
        heightScale = int(height / scale)
    else:
        scale = limited / minSize
        widthScale = int(width * scale)
        heightScale = int(height * scale)

    # 将两个值都除以64，然后取整,也就是说边长必须是64的倍数
    widthScale = int(widthScale/64) * 64
    heightScale = int(heightScale/64) * 64

    return widthScale, heightScale

def checkImageExsit(imageDir, imageName):
    for file in os.listdir(imageDir):
        if file.startswith(imageName):
            return True
    return False


def queue_prompt(prompt):
    while True:
        try:
            p = {"prompt": prompt, "client_id": client_id}
            data = json.dumps(p).encode('utf-8')
            req = urllib.request.Request("http://{}/prompt".format(server_address), data=data)
            return json.loads(urllib.request.urlopen(req, timeout=5).read())
        except URLError:
            print("连接超时，正在重试...")

def get_history(prompt_id):
    with urllib.request.urlopen("http://{}/history/{}".format(server_address, prompt_id)) as response:
        return json.loads(response.read())

def get_image(filename, subfolder, folder_type):
    data = {"filename": filename, "subfolder": subfolder, "type": folder_type}
    url_values = urllib.parse.urlencode(data)
    with urllib.request.urlopen("http://{}/view?{}".format(server_address, url_values)) as response:
        return response.read()

def handle_whitespace(string: str):
    return string.strip().replace("\n", " ").replace("\r", " ").replace("\t", " ")

def parse_name(ckpt_name):
    path = ckpt_name
    filename = path.split("/")[-1]
    filename = filename.split(".")[:-1]
    filename = ".".join(filename)
    return filename


def calculate_sha256(file_path):
    sha256_hash = hashlib.sha256()

    with open(file_path, "rb") as f:
        # Read the file in chunks to avoid loading the entire file into memory
        for byte_block in iter(lambda: f.read(4096), b""):
            sha256_hash.update(byte_block)

    return sha256_hash.hexdigest()

def get_images(ws, prompt):
    prompt_id = queue_prompt(prompt)['prompt_id']
    output_images = {}
    while True:
        out = ws.recv()
        if isinstance(out, str):
            message = json.loads(out)
            if message['type'] == 'executing':
                data = message['data']
                if data['node'] is None and data['prompt_id'] == prompt_id:
                    break #Execution is done
        else:
            continue #previews are binary data

    history = get_history(prompt_id)[prompt_id]
    for o in history['outputs']:
        for node_id in history['outputs']:
            node_output = history['outputs'][node_id]
            if 'images' in node_output:
                images_output = []
                for image in node_output['images']:
                    image_data = get_image(image['filename'], image['subfolder'], image['type'])
                    images_output.append(image_data)
                output_images[node_id] = images_output

    return output_images
    
def saveImages(result, imageDir,imageName,comment,previewImage=0): 
    metadata = PngInfo()
    metadata.add_text("parameters", comment)  
    k = 0 
    for node_id in result:    
        if k < previewImage:
            k += 1
            continue    
        for image_data in result[node_id]:            
            image = Image.open(io.BytesIO(image_data))
            imagePath = imageDir + "/" + imageName + f"_{k}" + ".png"
            image.save(imagePath,pnginfo=metadata) 
            k += 1
    

In [5]:
# 子文件夹
subDir = "FreeUTest"

# prompts文件
promptsFile = '../descriptions.txt'
with open(promptsFile, encoding='utf-8') as f:
    prompt_list = f.readlines()

# 设置图片保存路径
imageDir = f"f:/Poledriver/Remake/{subDir}"
# 如果文件夹不存在，创建文件夹
if not os.path.exists(imageDir):
    os.mkdir(imageDir)

In [6]:

from PIL import Image
from sdparsers import ParserManager
import websocket
import itertools

ws = websocket.WebSocket()
ws.connect("ws://{}/ws?clientId={}".format(server_address, client_id))

batch_size = 1
b1_list =[1,1.1,1.2]
b2_list =[1.2,1.4,1.6]
s1_list =[0.3,0.5,0.7,0.9]
s2_list =[0.2,0.4,0.6,0.8]

params = itertools.product(b1_list,b2_list,s1_list,s2_list)

prompt_string = f"analog film grain,bokeh,(TOK:1.5), A woman in a bridal gown walks down a flower-filled aisle, sunlight pouring through stained glass windows, surrounding her in ethereal radiance, skin pores,she have black long hair,big breasts,big ass," 

prompt['48']['inputs']['text'] = prompt_string        

# 读取原图片名字，去掉后缀.png
originalImagName = f"奶茶多多"

# 下面是确定实际生成图片的大小，种子，以及保存图片信息
# 保存图片的信息使用的是38号节点，也就是Save Image w/Metadata节点 
prompt['36']['inputs']['batch_size'] = batch_size
steps = 50
cfg = 5
sampler_name = prompt['51']['inputs']['sampler_name']
scheduler = prompt['51']['inputs']['scheduler']
positive = prompt['48']['inputs']['text']
negative = prompt['49']['inputs']['text']

modelname = prompt['4']['inputs']['ckpt_name']
basemodelname = parse_name(modelname)        
ckpt_path = "e:/sdwebui/webui/models/Stable-diffusion/" + modelname
modelhash = calculate_sha256(ckpt_path)[:10]

seed = random.randint(0, 976242998978323)
prompt['61']['inputs']['seed']= seed
count = 0
for param in params:
    b1 = param[0]
    b2 = param[1]
    s1 = param[2]
    s2 = param[3]
    prompt['58']['inputs']['b1'] = b1
    prompt['58']['inputs']['b2'] = b2
    prompt['58']['inputs']['s1'] = s1
    prompt['58']['inputs']['s2'] = s2
    prompt['59']['inputs']['b1'] = b1
    prompt['59']['inputs']['b2'] = b2
    prompt['59']['inputs']['s1'] = s1
    prompt['59']['inputs']['s2'] = s2

    comment = f"{handle_whitespace(positive)}\nNegative prompt: {handle_whitespace(negative)}\nSteps: {steps}, Sampler: {sampler_name}{f'_{scheduler}' if scheduler != 'normal' else ''}, CFG Scale: {cfg}, Seed: {seed}, Size: {896}x{1152}, Model hash: {modelhash}, Model: {basemodelname}, FreeU:({b1},{b2},{s1},{s2}),Version: ComfyUI"        

    print(f"生成图片的信息是：{comment}")
    file_name = f"{originalImagName}_{count}"
    
    # 连接api，生成并保存图片
    start_time = time.time()
    
    images = get_images(ws, prompt)   
    saveImages(images, imageDir, file_name, comment=comment,previewImage=0)     
    end_time = time.time()
    print(f"耗时{end_time-start_time}秒")
    count += 1
            

WebSocketAddressException: [Errno 11001] getaddrinfo failed